# Swarm 으로 챗봇 만들기

## 준비

In [5]:
## 사전 준비
!pip install git+https://github.com/openai/swarm.git
!pip install gradio duckduckgo_search

  Cloning https://github.com/openai/swarm.git to c:\users\inkyu.choi\appdata\local\temp\pip-req-build-ngt_790p
  Resolved https://github.com/openai/swarm.git to commit 0c82d7d868bb8e2d380dfd2a319b5c3a1f4c0cb9
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git 'C:\Users\inkyu.choi\AppData\Local\Temp\pip-req-build-ngt_790p'


In [6]:
import os

def load_api_keys(filepath):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

filepath = '../.env'

# API 키 로드 및 환경변수 설정
load_api_keys(filepath)

In [7]:
ollama_model = 'qwen3:latest'

In [17]:
from swarm import Swarm, Agent

client = Swarm()

agent = Agent(
    name="Document Summary Expert",
    instructions="""
    너는 문서를 요약하는 전문가야.
    주어진 문서를 핵심 키워드를 포함해서 요약해줘.
    반드시 한국어로만 응답해야 되며 한 줄 텍스트로 보여줘.

    응답예시:
    사용자가 정식 프로그램(데몬툴즈, 유니락, 엑스 플레그 등)도 PUA(잠재적으로 불필요한 앱) 또는 바이러스로 분류받는 현상에 대해 혼란스러워합니다.
    """,
    # functions=[get_news_articles],
    model=ollama_model
)

response = client.run(
    agent=agent,
    messages=[{"role": "user", "content": f"""중요도 낮음의 PUA로 시작하는 바이러스가 옛날파일들한테 나오는게 문제인데요.

물론 어릴적 받았었던 불법복제 게임에서도 나왔지만 정식 프로그램에도 나와서 혼란스럽습니다.

우선 나온목록이

PUA:Win32/Niguide

2014년도경에 네이버 소프트웨어 자료실에서 받은 데몬툴즈 설치파일.. 우선 네이버 자료실에서 받은 정식버전이구요.

PUA:Win32/CandyOpen

utorrent.exe 파일로 이것도 토렌트 사이트에서 받은 정식 토렌트실행기 파일입니다.

PUA:Win32/Vigua.A

agth 라고 외국게임 번역해주는 프로그램인데 걍 일반인이 무료로 푼거라 애매하네요.

PUA:Win32/Presenoker

이건 여러개 나왔는데 불법게임인 워크래프트3에 유명한 툴인 m16tool.exe 에서 하나, 어디서 받았는지 모르는 롤러코스터관련 reg.exe파일, 그리고 이건 정식 홈페이지에서 무료다운한 unlocker 라는 프로그램에서 나왔습니다.

우선 데몬툴즈, unlocker는 정식이라서, 워크래프트3의 m16tool는 워3 한번해봤다하면 너무 유명한거라.. 그리고 agth는 아직 버리기 아까워서 실행은 못시키고 제외설정에 넣어서 삭제 안시켰구요.

토렌트은 필요없고 롤러코스터는 직접산것도 있고 수상하기도 해서 제거시켰습니다.

근데 정식파일도 있고 그런데 바이러스 취급을 왜하는지... 굳이 신경써야하는건지 다 삭제해야하는지! 궁금하네요."""}],
)


In [22]:
res_list = response.messages[-1]['content'].split('\n</think>')
res_list = res_list[-1].strip()

res_list

'사용자는 정식 프로그램(데몬툴즈, uTorrent, Unlocker 등)도 PUA 또는 바이러스로 오인받는 현상에 대해 혼란스러워하며, 이는 false positive 가능성이나 악성코드 탐지 오류로 인한 것으로, 파일 출처와 동작 방식을 확인해 추가 조치가 필요하다고 판단합니다.'